## Import necessary modules
Run this cell before running any other cells

In [2]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

<hr>

# BLE
## ArtemisBLEController
The class **ArtemisBLEController** (defined in *ble.py*) provides member functions to handle various BLE operations to send and receive data to/from the Artemis board, provided the accompanying Arduino sketch is running on the Artemis board. <br>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reload_config()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reload changes made in <em>connection.yaml.</em></span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">connect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Connect to the Artemis board, whose MAC address is specified in <em>connection.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">disconnect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Disconnect from the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">is_connected()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Return a boolean indicating whether your controller is connected to the Artemis board or not.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">send_command(cmd_type, data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Send the command <strong>cmd_type</strong> (integer) with <strong>data</strong> (string) to the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">receive_float(uuid) <br> receive_string(uuid) <br> receive_int(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Read the GATT characteristic (specified by its <strong>uuid</strong>) of type float, string or int. <br> The type of the GATT
            characteristic is determined by the classes BLEFloatCharacteristic, BLECStringCharacteristic or
            BLEIntCharacteristic in the Arduino sketch.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">start_notify(uuid, notification_handler)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Activate notifications on the GATT characteristic (specified by its <strong>uuid</strong>). <br> <strong>notification_handler</strong> is a
            function callback which must accept two inputs; the first will be a uuid string object and the second will
            be the bytearray of the characteristic value.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">bytearray_to_float(byte_array) <br> bytearray_to_string(byte_array) <br> bytearray_to_int(byte_array)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Convert the <strong>bytearray</strong> to float, string or int, respectively. <br> You may use these functions inside your
            notification callback function.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">stop_notify(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Stop notifications on the GATT characteristic (specified by its <strong>uuid</strong>).</span></th>
    </tr>
</table>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Variables</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">uuid</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A dictionary that stores the UUIDs of the various characteristics specified in <em>connection.yaml</em>.</span></th>
    </tr>
</table>

## Configuration
- The MAC address, Service UUID and GATT characteristic UUIDs are defined in the file: *connection.yaml*.
- They should match the UUIDs used in the Arduino sketch.
- The artemis board running the base code should display its MAC address in the serial monitor.
- Update the **artemis_address** in *connection.yaml*, accordingly.
- Make sure to call **ble.reload_config()** or **get_ble_controller()** (which internally calls **reload_config()**) after making any changes to your configuration file.

<hr>

In the below cell, we create an **ArtemisBLEController** object using **get_ble_controller()** (defined in *ble.py*), which creates and/or returns a single instance of **ArtemisBLEController**. <br>
<span style="color:rgb(240,50,50)"> __NOTE__: Do not use the class directly to instantiate an object. </span><br>

In [3]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2025-02-04 16:31:55,967 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:81:80:25:12:64
2025-02-04 16:32:00,492 | INFO     |: Connected to c0:81:80:25:12:64


C:\Users\zhang\Classes\ECE_4160\labs\ble_robot_1.2\ble_python\base_ble.py:104: FutureWarning: This method will be removed future version, pass the callback to the BleakClient constructor instead.
  self.client.set_disconnected_callback(self.disconnect_handler)


In [4]:
timestamps = []
temperatures = []
def handle_notice(uuid, byte_array):
    string = byte_array.decode("utf-8") 
    timestamps.append(string[:string.index("|")])
    temperatures.append(string[string.index("|")+1:])

In [5]:
ble.start_notify(ble.uuid['RX_STRING'], handle_notice)


In [6]:
ble.send_command(CMD.POPULATE_TIME_DATA, "")
ble.send_command(CMD.SEND_TIME_DATA, "")


In [7]:
print(timestamps)
print(temperatures)

['Time:73029', 'Time:73030', 'Time:73030', 'Time:73030', 'Time:73031', 'Time:73031', 'Time:73031', 'Time:73031', 'Time:73032', 'Time:73032', 'Time:73032', 'Time:73032', 'Time:73033', 'Time:73033', 'Time:73033', 'Time:73033', 'Time:73033', 'Time:73034', 'Time:73034', 'Time:73034', 'Time:73034', 'Time:73035', 'Time:73038', 'Time:73038', 'Time:73038', 'Time:73039', 'Time:73039', 'Time:73039', 'Time:73039', 'Time:73039', 'Time:73040', 'Time:73040', 'Time:73040', 'Time:73040', 'Time:73041', 'Time:73041', 'Time:73041', 'Time:73041', 'Time:73042', 'Time:73042', 'Time:73042', 'Time:73042', 'Time:73042', 'Time:73043', 'Time:73043', 'Time:73043', 'Time:73043', 'Time:73044', 'Time:73045', 'Time:73045']
['Temp:70', 'Temp:69', 'Temp:70', 'Temp:68', 'Temp:70', 'Temp:70', 'Temp:70', 'Temp:70', 'Temp:69', 'Temp:69', 'Temp:70', 'Temp:71', 'Temp:70', 'Temp:71', 'Temp:70', 'Temp:70', 'Temp:70', 'Temp:70', 'Temp:70', 'Temp:70', 'Temp:69', 'Temp:69', 'Temp:69', 'Temp:70', 'Temp:70', 'Temp:70', 'Temp:70', '

In [11]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

T:1147802


In [12]:
ble.send_command(CMD.SEND_TWO_INTS, "2|-6")

## Disconnect

In [9]:
# Disconnect
ble.disconnect()

2025-01-29 15:11:34,143 | INFO     |: Disconnected from C0:81:80:25:12:64
2025-01-29 15:11:34,143 | INFO     |: Disconnected from C0:81:80:25:12:64
